## Computational Chemistry for Experimentalists
## Chapter 20: Excited Electronic States and UV/Vis Absorption

Excited electronic states are key to UV/vis absorption and photochemistry. Computing excited states is a very important use case for computational quantum chemistry methods. This set of examples illustrates some successes and challenges of excited-state modeling. 

In [ ]:
import tabulate
from pyscf import gto,scf,tdscf,dft,tddft
from pyscf.tools import cubegen 
import numpy 
import matplotlib.pyplot as plt

## Part 1: Li atom s->p excitation: Linear response TDHF vs. orbital energy differences

Molecular orbital energy diagrams are often used to illustrate excited states and photon absorption. However, mean-field Hartree-Fock orbital energies do not and should not perfectly reproduce excitation energies, just as Hartree-Fock total energies do not perfectly reproduce ground-state properties. Linear response time-dependent density functional theory is usually more accurate in practice. LR-TDDFT excitation energies include orbital energies as part of the calculation of a molecule's response to an applied perturbation. 

This section considers the optically allowed s-p excitation of an isolated Li atom. The atom has an experimental gas-phase UV-vis absorbance 1.847 eV for 1s2 2s->1s2 2p excitation. We'll do these calcualtions near the basis set limit. 

In [ ]:
m=gto.Mole(atom='Li',basis='aug-cc-pvqz',spin=1)
m.build()
mf=scf.ROHF(m)
mf.kernel()
moe=mf.mo_energy
mt = tdscf.TDHF(mf)
mt.nstates = 10
mt.kernel()
mt.analyze()
#print(dir(mt))
print('Experimental 2s-2p energy difference: 1.847 eV')
print('2s-2p orbital energy difference:      %5.3f eV'%(27.211*(moe[2]-moe[1])))
print('2s-2p excitation:                     %5.3f eV'%(27.211*(mt.e[0])))

In [ ]:
# Here's how to access excited state energies and oscillator strengths 
print(mt.e)
print(mt.oscillator_strength(gauge='length'))
print(mt.oscillator_strength(gauge='velocity'))

## Part 2: Dipole-Allowed and Higher-Order Transitions 

In the limit of a very long wavelength, a ground-excited state transition depends only on the change of electric dipole moment between ground and excited states. At shorter wavelengths, magnetic dipole and electric quadrupole (and other) transitions become important. These are the transitions responsible for circular dichromism and optical activity. Here we compute the H atom 1s-2s  absorbance spectrum using electric dipole and quadrupole terms. 

In [ ]:
m=gto.Mole(atom='H',spin=1,basis='cc-pvqz')
m.build()
mf=scf.ROHF(m)
mf.kernel()
moe=mf.mo_energy
mt = tdscf.TDHF(mf)
mt.nstates = 1
mt.kernel()
print(mt.transition_dipole())
print(mt.transition_quadrupole())
v0=mt.oscillator_strength(gauge='velocity',order=0)
v1=mt.oscillator_strength(gauge='velocity',order=1)
v2=mt.oscillator_strength(gauge='velocity',order=2)
v3=mt.oscillator_strength(gauge='velocity',order=3)
es=27.211*mt.e

tabulate.tabulate(numpy.transpose([es,v0,v1,v2,v3]),tablefmt='html')

## Part 3: From Absorbance Peaks to Computed Spectra

True absorbance spectra are not simply electronic excitations. A much better model considers transitions between vibrational levels in teh ground and excited states. To simplify interpretation, we can add a  Gaussian broadening to each computed electronic transition, and compute the total absorption at each wavelength

In [ ]:
def spec(photonwavelengths,energies,strengths,broadEV=.1,norm=True):
    broad=broadEV/27.211# Broadening in Hartree 
    absorbances=numpy.zeros_like(photonwavelengths) # Output absorbances 
    photonenergies=45.5640/photonwavelengths # Photon energy in Hartree for each wavelength E=hv=hc/lambda 
    statewavelengths=45.5640/energies
    print('Expected absorbance peaks (nm): ',statewavelengths)
    print('Their relative strengths:',strengths/max(strengths))
    for j in range(len(energies)):
        facs=numpy.exp(-1.0*(photonenergies-energies[j])**2/broad)
        absorbances = absorbances + strengths[j]*facs
    if(norm):
        absorbances=absorbances/max(absorbances)
    return(absorbances)
    

In [ ]:
m=gto.Mole(atom='C 0.0 0.0 0.0; O 0.0 0.0 1.128',basis='cc-pvtz')
m.build()
mf=scf.RHF(m)
mf.kernel()
mft=tdscf.TDHF(mf)
mft.nstates = 10
mft.kernel()
v=mft.oscillator_strength(gauge='velocity')



In [ ]:
wavelengths=numpy.array(list(numpy.arange(50,250,.1)))
absorbances=spec(wavelengths,mft.e,v,broadEV=.05)
plt.plot(wavelengths,absorbances)
plt.xlabel('Wavelength (nm)')
plt.ylabel('Relative absorption')
plt.show()

## Part 4: Method Dependence 

Occupied-virtual energy differences and excited states strongly depend on the method used. Here we consider the experimental UV absorbance of carbon monoxide CO. CO has two triplet excited states 3Pi and 3Sigma at 6.04 eV (205 nm) and 6.92 eV (179 nm) above the ground state, and an optically accessible singlet excited state 1Pi at 8.07 eV (153 nm) above the ground state. These values are form the NIST Chemistry WebBook, at the bottom of the table of excited state energies T_e, converted from cm-1. 

In [ ]:
meths=['HF,LYP','B3LYP','BLYP','LDA,VWN']
wavelengths=numpy.array(list(numpy.arange(50,250,.1)))
absorbances=[]
for meth in meths:
    md=dft.RKS(m,xc=meth)
    md.kernel()
    mdt=tddft.TDDFT(md)
    mdt.nstates = 8
    mdt.kernel()
    v=mdt.oscillator_strength(gauge='velocity')
    a=spec(wavelengths,mdt.e,v,broadEV=.05,norm=False)
    absorbances.append(a)
absorbances=numpy.array(absorbances)
absorbances=absorbances/numpy.max(absorbances)



In [ ]:
for i in range(len(meths)):
    plt.plot(wavelengths,absorbances[i],label=meths[i])
plt.xlabel('Wavelength (nm)')
plt.ylabel('Relative absorption')
plt.legend()
plt.show()



# Practice Problems

Compute the Hartree-Fock, B3LYP, and BLYP UV/vis absorbance spectrum of p-benzoquinone, using some reasonable guess for the initial geometry. Which computed absorbance spectrum best corresponds to the molecule's observed yellow color? 